# Collaborative filtering model

In [2]:
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
from implicit.nearest_neighbours import bm25_weight
from implicit.als import AlternatingLeastSquares
import os

In [3]:
materials_root = "../materials"
working_root = "../working"

## Load data

**Load `userid-artist-counts.csv`** for training the model.

In [4]:
data = pd.read_csv(os.path.join(working_root, "userid-artist-counts.csv"))
data.head()

,userid,artist_mbids,count
0,1,002e9f6e-13af-4347-83c5-f5ace70e0ec4,1
1,1,01252145-c9e8-4de5-a480-9b2bed05450a,14
2,1,02a7de68-2681-4d7e-8c36-6f2fdb37c07d,1
3,1,0383dadf-2a4e-4d10-a46a-e9e041da8eb3,12
4,1,03f93de6-6d62-4710-bcc7-9b3d7c3d95f5,1


**Load `musicbrainz_artist.csv` for `artist_mbids` to `name` mapping.**

In [5]:
musicbrainz_artist = pd.read_csv(os.path.join(materials_root, "musicbrainz_artist.csv"))
musicbrainz_artist.set_index("artist_mbid", inplace=True)
musicbrainz_artist.head()

,name
artist_mbid,
fadeb38c-833f-40bc-9d8c-a6383b38b1be,Доктор Сатана
49add228-eac5-4de8-836c-d75cde7369c3,Pete Moutso
165a49a0-2b3b-4078-a3c1-905afdc07c0a,Babyglock
7b4a548e-a01a-49b7-82e7-b49efeb9732c,Aric Leavitt
60aca66f-e91a-4cb5-9308-b6e293cd833e,Fonograff


**Process the date for training**

In [6]:
users = data["userid"].astype("category")
artists = data["artist_mbids"].astype("category")
plays = coo_matrix(
        (data["count"].astype(np.float32),
        (
            users.cat.codes.copy(),
            artists.cat.codes.copy()
        ),
    )
).tocsr()

artist_user_plays = bm25_weight(plays, K1=100, B=0.8)
user_plays = artist_user_plays.T.tocsr()

## Model training

In [7]:
model = AlternatingLeastSquares(factors=64, regularization=0.05, alpha=2.0)
model.fit(artist_user_plays.tocsr())

100%|███████████████████████████████████████████████████████████████████████████████████| 15/15 [00:02<00:00,  7.11it/s]


## Generate recommendations

**Recommendation example 1**

In [8]:
userid = 123
ids, scores = model.recommend(userid, user_plays[userid], N=10)

In [9]:
print(f'Recommendations for user: {userid}')
recommendation = pd.DataFrame({"artist": musicbrainz_artist.loc[artists[ids]]['name'].tolist(), "score": scores})
recommendation.reset_index(drop=True, inplace=True)
recommendation

Recommendations for user: 123


,artist,score
0,Laurie Stras,0.945264
1,Heart,0.921970
2,Fiona Joy Hawkins,0.878139
3,The Felice Brothers,0.877666
4,上海アリス幻樂団,0.876827
5,Victor Ruiz,0.868608
6,MEUTE,0.843488
7,Lit,0.841911
8,Light Up the Sky,0.817305
9,Xperiment,0.817300


**Recommendation example 2**

In [10]:
userid = 1234
ids, scores = model.recommend(userid, user_plays[userid], N=10)

In [11]:
print(f'Recommendations for user: {userid}')
recommendation = pd.DataFrame({"artist": musicbrainz_artist.loc[artists[ids]]['name'].tolist(), "score": scores})
recommendation.reset_index(drop=True, inplace=True)
recommendation

Recommendations for user: 1234


,artist,score
0,Jeuce,0.973729
1,The National,0.968783
2,Survivor,0.962908
3,Sybreed,0.932940
4,Beastie Boys,0.911920
5,Freya Ridings,0.911554
6,bollywoodFM,0.899826
7,Joe Cocker,0.893450
8,Nine Black Alps,0.889453
9,The Mission,0.887710


## Find similar artists

**Similar artists example 1**

In [12]:
artistid = "03f93de6-6d62-4710-bcc7-9b3d7c3d95f5"
artistname = musicbrainz_artist.loc[artistid]['name']
artist_idx = artists.index[artists == artistid].tolist()[0]

In [13]:
ids, scores = model.similar_items(artist_idx, N=10)

In [14]:
print(f'Artists similar to {artistname}')
similar_artists = pd.DataFrame({"artist": musicbrainz_artist.loc[artists[ids]]['name'].tolist(), "score": scores})
similar_artists.reset_index(drop=True, inplace=True)
similar_artists

Artists similar to Z‐Trip


,artist,score
0,Peter Pan Speedrock,1.0
1,Johnossi,1.0
2,Peter Cetera,1.0
3,IU,1.0
4,Nubya Garcia,1.0
5,tUnE-yArDs,1.0
6,Immolation,1.0
7,Terence Blanchard,1.0
8,Big Black Delta,1.0
9,Ex Deo,1.0


**Similar artists example 2**

In [15]:
artistid = "8f92558c-2baa-4758-8c38-615519e9deda"
artistname = musicbrainz_artist.loc[artistid]['name']
artist_idx = artists.index[artists == artistid].tolist()[0]

In [16]:
ids, scores = model.similar_items(artist_idx, N=10)

In [17]:
print(f'Artists similar to {artistname}')
similar_artists = pd.DataFrame({"artist": musicbrainz_artist.loc[artists[ids]]['name'].tolist(), "score": scores})
similar_artists.reset_index(drop=True, inplace=True)
similar_artists

Artists similar to The Clash


,artist,score
0,Labrinth,1.0
1,Fleet Foxes,1.0
2,Evan Seleven,1.0
3,The Clash,1.0
4,Sleeck,1.0
5,WHITEFISTS,1.0
6,Peggy Gou,1.0
7,Endel,1.0
8,Simple Minds,1.0
9,OceanLab,1.0
